# 基于矩阵分解的图机器学习模型

## 1. Introduction
极大的受到Dimensionality Reduction思想的影响，在图机器学习最早期的研究中涌现了一批基于Matrix Factorization的模型。这类模型往往以重建图的邻接矩阵为目标，学习各个节点的表示。尽管从今天的眼光来看，这些模型有的较大的局限性，但事实上它们的思想依然启发着许多如今的研究。

在这篇文章中，我们将介绍其中三篇比较有代表性的工作，包括Graph Factorization$^{[1]}$，GraRep$^{[2]}$以及HOPE$^{[3]}$。同时，我们也提供了这三个模型的Pytorch Demo。因笔者能力有限，若有谬误，请不吝指正！

## 2. Graph Factorization
GF$^{[1]}$是Google在13年提出的一篇文章，可以说是最早的图机器学习模型之一。它的思想在今天看来十分简单粗暴。

### 2.1 Objective Function


我们直接上目标函数：

$$\mathcal{L}=\frac{1}{2} \sum_{(i, j) \in E}\left(y_{i j}-\left\langle h_{i}, h_{j}\right\rangle\right)^{2}+\frac{\lambda}{2} \sum_{i}\left\|h_{i}\right\|^{2},$$

其中，$y_{ij}$为节点$(i,j)$之间的边权重，$h_i$为节点$i$的表示，$h_j$为节点$j$的表示，$\lambda$为正则系数。在整篇专栏中，我们将尽可能统一各符号的含义，因此公示符号会和原论文中的略有不同。

显然，这是一个简单的$MSE$目标函数加上一个$L_2$正则项。该目标函数希望节点对的表示的内积可以重建$y_{ij}$。当$y_{ij}$为边权重时，该目标等同重建图的邻接矩阵。同时，该目标可以看作是一个关于$y_{ij}$的回归模型，因此这也往往要求$y_{ij}$是连续的，否则模型效果可能会有极大的损失。值得一提的是，尽管原论文中没有提到，但这里的$y_{ij}$也可以为任意的一维（连续）edge feature。

### 2.2 Graph Partitioning
这个损失函数的设计思路是显而易见的，但该模型存在一个显著的问题。即，当图中节点数量较多，并且嵌入的维度也较大时，我们可能很难将整张图存在单个机器的内存中。

显然，为了解决这一问题，一个思路就是将图数据分布式的存储在集群中，后进行分布式的训练。比如，我们可以直接利用流行的图分割算法，如METIS等，将图划分为互不重叠的多个簇。之后，集群中的一台机器仅存储一个簇的数据，并只更新该簇中节点的表达。然而，这个简单的思路可能会使簇中节点丢失部分一阶邻居，致使信息损失、模型效果下降。

那么一个直观的降低信息损失的思路就是：额外保留簇中各个节点存在于簇外的一阶邻居。倘若能做到这一点，那么基于上述目标函数进行优化时，就不会发生信息的丢失。基于这一思路，该文章提出了一个图分割算法，使得该优化过程scalable且无信息损失。这个图分割算法事实上才是这篇文章的主要贡献。

<img src="img/GraphPartitioning.jpg" width="200">

该文章提出的图分割算法结果如上图所示，经该算法分割后，每个簇包含两个部分，一个部分是与其它各个簇均不重合的节点$O_k \in V$，即上图颜色较亮的部分；另一部分是$B_k = \{ v \in V | v \notin O_k, \exists u \in O_k, (u,v) \in E \}$，即上图颜色较暗的部分，这一部分的节点会存在于多个簇中，目的是为了确保$O_k$中各个节点均能找到对应的一阶邻居。最终一台机器$k$会存储的节点的集合为$V_k = O_k \cup B_k$。作者们称$O_k$ is *owned* by machine $k$ and $B_k$ is *borrowed* by machine $k$，可以说是比较形象的。

然而，因为存在于$B_k$中的节点会存在于不同的簇中，所以它们的表达会在不同的簇中独立的被更新。这就可能会导致这些节点的表达难以收敛。那么一个直观的思路就是在损失函数$\mathcal{L}$中约束$\sum_{k=1}^{K} \sum_{i \in B_{k}}\left\|h_{i}-h_{i}^{(k)}\right\|^{2}=0$，其中$h_i$和$h_i^{(k)}$均是learnable parameters，但是$h_i^{(k)}$代表节点$i$在机器$k$上的“临时”表达，仅仅是辅助参数，而$h_i$才是节点$i$最终的表示。

这个约束可以促使各个机器上的$h_i^{(k)}$收敛。将该约束植入损失函数的一个直接的方法则是利用Lagrange Multiplier。最终该模型损失函数为：

$\begin{aligned}
    \mathcal{L} &= \sum_{k=1}^{K} \mathcal{L}_{k}+\frac{1}{2} \sum_{k=1}^{K} \left[ \mu \sum_{i \in B_{k}}\left\|h_{i}-h_{i}^{(k)}\right\|^{2} \right],\\
    \mathcal{L}_k &=\frac{1}{2} \sum_{(i, j) \in E, \atop i, j \in V_{k}}\left(y_{i j}-\left\langle h_{i}^{(k)}, h_{j}^{(k)}\right\rangle\right)^{2}+\frac{\lambda}{2} \sum_{i \in V_{k}}\left\|h_{i}^{(k)}\right\|^{2}.
\end{aligned}$

最后的问题则是，我们实现这个图分割算法呢？由上述思路我们可以看见，为了使$B_k$中节点的表达收敛，我们不得不在集群中传输$B_k$中节点的相关参数。倘若$\sum_k \left| B_k\right|$很大的话，那么机器之间数据交换的代价可能非常昂贵。因此，不同于旨在最小化edge cuts的主流图分割算法，该文章提出了一个最小化各个簇在簇外的一阶邻居数量的图分割算法。显然，最小化这个目标是一个NP-hard的问题，因此作者们提出了一个贪婪策略，使得$\sum_k \left| B_k\right|$尽可能的小，以提升算法的效率。该算法的具体流程，我们请读者参阅原论文Section 5.2.

### 2.3 Implementation
在[GF.py](https://github.com/siqim/Machine-Learning-with-Graphs/blob/main/examples/0_MatrixFactorization/GF.py)中，我们实现了GF，并且在[Wikipedia](https://snap.stanford.edu/node2vec/)和[ogbn-arxiv](https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-arxiv)数据集上评估了该模型实现。该模型实现可在Wikipedia数据集上取得约0.39的Mirco-F1，在ogbn-arxiv数据集上取得约0.51的Accuracy。我们对Wikipedia效果的评估希望同[cogdl leaderboard](https://github.com/THUDM/cogdl#unsupervised-multi-label-node-classification)尽可能接近，即，用90%的标注数据进行训练，用10%的标注数据进行测试。对ogbn-arxiv的评估则完全基于ogb给出的划分和evaluator。

这两个数据集的edge weight均为离散，且$e_{ij}\in \{0, 1\}$，所以文章中提出的目标函数较难取得体面的成绩。因此，我们的实现基于负采样来重建$e_{ij}$，以习得各个节点的表达。最后我们将习得的表达，拼接上各节点的特征（若有），利用FC层进行节点分类任务。

该模型在Wikipedia上的效果较为可观，但在ogbn-arxiv上的效果不如直接利用node feature训练的MLP，这可能是模型表达能力有限所导致的。


### 2.4 Summary
这篇文章总体来说非常的Google，很偏向落地应用。重点在于模型的大规模部署训练，而不是特定的目标函数设计。

事实上，ClusterGCN$^{[4]}$（也是一篇Google的文章）跟这篇文章有异曲同工之妙。不同的是，ClusterGCN基于GCN，簇间的信息损失远不止节点的部分一阶邻居而已，而倘若要维护各个簇在簇外的$k$阶邻居，那么代价可能又过高了。因此ClusterGCN的可扩展性建立在簇间的信息损失之上。

总体来说，该模型的优点有：

- 能够较好的适配新加入的节点
- 能够适配大规模图数据

该模型的局限性有：

- 往往仅适用于一维连续的$y_{ij}$，且仅能捕捉节点间的一阶关系
- 模型仅输入$(i,j) \in E$，故而缺少"负"样本，可能出现较多的"False Positives"
- Embedding lookup矩阵后直接跟损失函数，模型表达能力有限、节点间无参数共享
- 无法直接利用节点的特征

## 3. GraRep

GraRep$^{[3]}$是一篇15年的文章。这篇文章提出时DeepWalk$^{[5]}$和LINE$^{[6]}$正流行起来。然而DeepWalk这种直接基于skip-gram的算法，在训练时无法区别出一个节点的邻居的阶数：每个节点在学习自己的表示时，只能知道自己周围$k$阶邻域中包含哪些节点，而不知道这些节点具体是自己的几阶邻居。尽管LINE能够直接区分一阶和二阶邻居，但它无法容易的扩展到捕捉任意$k$阶邻居的信息上。

为了在表示学习中更好的区分节点不同阶的邻居，同时也能够扩展到捕捉任意阶的邻居上，研究者们提出了GraRep。


### 3.1 Overview of the Model
GraRep提出了一个将节点的$k$阶信息映射到不同的子空间上的算法。若希望捕捉最高$k$阶的信息，那么则分别计算$k$个模型，每个模型捕捉不同阶的信息，最后将各个模型习得的表达拼接，作为捕捉了一个节点的所有$k$阶信息的表达。

具体来说，该算法首先定义了一个*1-step probability transition matrix* $A$:

$$A = D^{-1}S,$$

其中，$S$是邻接矩阵，$D$是度矩阵，即：
$D_{i j}=\left\{\begin{array}{ll}\sum_{p} S_{i p}, & \text { if } i=j \\ 0, & \text { if } i \neq j\end{array}\right..$

在这个定义下，$A_{i,j}$即可认为是节点$i$经过一步转移到节点$j$上的概率，体现了节点对$(i,j)$之间的一阶连接关系。那么，如果我们能够重建$A$这个矩阵，我们的模型应当就能够捕捉到节点一阶邻居的信息。

在此基础上，作者们定义了*k-step probability transition matrix* $A^k$:

$$A^{k}=\underbrace{A \cdots A}_{k}.$$

显然，$A_{i,j}^k$即可认为是节点$i$经过$k$步转移到节点$j$上的概率，体现了节点对$(i,j)$之间的$k$阶连接关系。从条件概率的角度来看，我们也可以认为$A^k_{i,j} = p_k(j \mid i)$。因此，如果我们能够重建$A^k$这个矩阵，我们的模型应当就能够捕捉到节点$k$阶邻居的信息。

接下来的问题就是，我们如何重建$A^k$呢？让我们首先为节点$i$定义两组向量$\vec{w_i}$以及$\vec{c_i}$，其中$\vec{w_i}$是节点$i$为源节点时的表达，$\vec{c_i}$是节点$i$为其它节点的上下文节点时的表达。在这个定义下，若$(i,j)$之间存在长度为$k$的路径，我们就希望最大化$\sigma(\vec{w_i} \cdot \vec{c_j})$，否则就最小化$\sigma(\vec{w_i} \cdot \vec{c_j})$。这里我们称节点$i$为源节点(source node)，称节点$j$是节点$i$的上下文节点(context node)，且$\sigma(\cdot)$指*sigmoid*函数。

为了达到上述目标，在学习节点的$k$阶表达时，给定节点$i$，作者们极大的受到[7]和[8]的启发，提出了最大化以下目标函数：

$$J_{k}(i)= \left(\sum_{j \in V} p_{k}(j \mid i) \log \sigma(\vec{w_i} \cdot \vec{c_j})\right) +   \lambda \mathbb{E}_{j^{\prime} \sim p_{k}(V)}\left[\log \sigma(-\vec{w_i} \cdot \vec{c_{j^{\prime}}})\right],$$

其中，$\lambda$是负采样系数，$j^{\prime}$是根据分布$p_k(V)$采样得到的负样本。$p_k(j^{\prime}) := \frac{1}{N} \sum_{i} A_{i, j^{\prime}}^{k},$ 即从任意点出发$k$步后以$j^{\prime}$为终点的概率。


最后，从整张图来看，目标函数即是：

$$J_k = \sum_{i \in V} J_k(i).$$

随后我们则可以通过优化算法，对$\{ J_k \mid k \in \{1, \dots, K\} \}$分别进行优化，之后将求得的各阶的表达拼接，作为一个节点最终的表达，即$w_i = \|_1^K w^{(k)}_i .$


### 3.2 Derivation and Optimization
剩下的问题就是，$J_{k}(i)$这个目标函数是怎么得到的，我们又该怎样优化它呢？

首先我们尝试回答第一个问题，即$J_{k}(i)$是如何得到的。作者们在文章中并没有对提出的损失函数进行推导，只说了这个损失是由*noise contrastive estimation* (NCE)$^{[7]}$定义的。我们可以将[7]中的(8)式改写如下：

$$J(i) = \mathbb{E}_{j \sim P(j \mid i)}\left[\log \sigma\left(\Delta s(i, j)\right)\right]+ \lambda \mathbb{E}_{j \sim P_{n}(j)}\left[\log \left(1-\sigma\left(\Delta s(i, j)\right)\right)\right],$$

其中，$\Delta s(i, j) = \vec{w_i} \cdot \vec{c_j} -\log \left(\lambda P_{n}(j)\right).$

若我们假设$\log \left(\lambda P_{n}(j)\right) = 0$，并将$J(i)$第一项的期望展开，即可得到GraRep中的$J_k(i)$。假设$\log \left(\lambda P_{n}(j)\right) = 0$并不是少见的，这其中牵扯到NCE和Negative Sampling (NS)的联系。关于NCE，我们请读者参阅[paper1](https://papers.nips.cc/paper/2013/file/db2b4182156b2f1f817860ac9f409ad7-Paper.pdf])和[blog1](https://leimao.github.io/article/Noise-Contrastive-Estimation/)；关于NS，我们请读者参阅[paper2](https://arxiv.org/pdf/1402.3722.pdf)；关于NCE和NS的区别与联系，我们请读者参阅[blog2](https://leimao.github.io/article/Word2Vec-Classic/)。

简单来说，NS可以看作是NCE简化后的一个特例，NS和NCE具体哪一个效果更好也很难说。GraRep中提出的目标函数$J_k(i)$更像是NS和NCE的杂糅。如果把$J_k(i)$中的条件概率拿掉，它是不是就和word2vec中的负采样非常相似了呢？有趣的是，当我们利用作者提出的目标函数优化模型时，在Wikipedia数据集中我们可以取得约0.55的Mirco-F1，在ogbn-arxiv数据集上可以取得约0.62的Accuracy；当我们在推导中去掉条件概率，使之完全成为负采样目标函数时，我们可以在Wikipedia数据集中取得约0.59的Micro-F1，在ogbn-arxiv数据集上可以取得约0.61的Accuracy。

那么最后一个问题就是，GreRep是如何优化这个目标函数的呢？它的优化方法又有什么特殊之处呢？除此之外，到这里我们可能也会想，GraRep到底和矩阵分解有什么关联，为什么要将它划分为这一类模型呢？答案就是它所采取的的优化方法并非是梯度下降，而是将该问题转化为了一个矩阵分解问题进行求解。

GraRep的优化方法和它的目标函数均极大的受到[8]的影响。[8]这篇文章将基于负采样的skip-gram模型建模成了一个矩阵分解问题，是很有意思的一篇文章。GraRep基本上照搬了[8]中的思路，具体来说，现在我们有：

$$J_{k}(i)= \left(\sum_{j \in V} p_{k}(j \mid i) \log \sigma(\vec{w_i} \cdot \vec{c_j})\right) +   \lambda \mathbb{E}_{j^{\prime} \sim p_{k}(V)}\left[\log \sigma(-\vec{w_i} \cdot \vec{c_{j^{\prime}}})\right].$$

我们将上述第二项中的期望展开，可以得到：


$$\mathbb{E}_{j^{\prime} \sim p_{k}(V)}\left[\log \sigma(-\vec{w_i} \cdot \vec{c_{j^{\prime}}})\right] =p_{k}(j) \cdot \log \sigma(-\vec{w_i} \cdot \vec{c_j})+\sum_{j^{\prime} \in V \backslash\{j\}} p_{k}\left(j^{\prime}\right) \cdot \log \sigma(-\vec{w_i} \cdot \vec{c_{j^{\prime}}}),$$

其中，$p_{k}(j)=\sum_{i} q(i) p_{k}(j \mid i)，$ $q(i)$是节点$i$被采样为源节点的概率，可设为$q(i) = \frac{1}{N}$。 最终可以得到 $p_k(j) =\frac{1}{N} \sum_{i} A_{i, j}^{k}.$

因此，若给定某一节点对$(i,j)$，局部目标 (local objective) $J_k(i,j)$可以写为：

$\begin{aligned}
J_{k}(i, j) &= p_{k}(j \mid i) \cdot \log \sigma(\vec{w_i} \cdot \vec{c_j})+\lambda \cdot p_{k}(j) \cdot \log \sigma(-\vec{w_i} \cdot \vec{c_j}) \\
&=A_{i, j}^{k} \cdot \log \sigma(\vec{w_i} \cdot \vec{c_j})+\frac{\lambda}{N} \sum_{i^{\prime}} A_{i^{\prime}, j}^{k} \cdot \log \sigma(-\vec{w_i} \cdot \vec{c_j})
\end{aligned}$


若假设我们有足够大的表示维度$d$，那么我们可以认为每一对$\vec{w_i} \cdot \vec{c_j}$仅负责"重建"$A^k$中的一个元素$A_{i,j}^k$，也就是说每一对$\vec{w_i} \cdot \vec{c_j}$与其它节点对的点积均互相独立。这就意味着为了获取最优的$\vec{w_i}$和$\vec{c_j}$，我们仅需要对每对节点$(i,j)$单独优化$J_k(i,j)$，来得到最优的表达。所以，我们现在希望对$J_k(i,j)$这个局部目标进行优化。

我们首先将$w_i \cdot c_j$看作一个整体，并令$x = \vec{w_i} \cdot \vec{c_j}$。接着我们令$\frac{\partial J_{k}(i,j)}{\partial x}=0$，我们会发现$x$仅有一个合法解：

$$x = \vec{w} \cdot \vec{c}=\log \left(\frac{A_{i, j}^{k}}{\sum_{i^{\prime}} A_{i^{\prime}, j}^{k}}\right)-\log (\frac{\lambda}{N}).$$

这就意味着当且仅当$\vec{w} \cdot \vec{c}$等于上式时$J_k(i,j)$为最优。

因此，我们可以首先算出一个矩阵$Y^k$，使其中每一个元素$Y^k_{i,j} = \log \left(\frac{A_{i, j}^{k}}{\sum_{i^{\prime}} A_{i^{\prime}, j}^{k}}\right)-\log (\frac{\lambda}{N})$。随后我们可以再将$Y^k$分解为两个矩阵，使得$Y^k = W^k C^k$，那么我们就可以得到使得$J_k(i,j)$最优的$\vec{w_i}$和$\vec{c_j}$，它们分别是$W^k$矩阵的第$i$行和$C^k$矩阵的第$j$列。

最后就是我们该如何分解$Y^k$这个矩阵。显然，我们可以利用low-rank SVD来分解$Y^k$，所取的阶数$d$即为我们表示的维度。具体来说，经过low-rank SVD后，我们可以得到：

$$Y^{k} \approx Y_{d}^{k}=U_{d}^{k} \Sigma_{d}^{k}\left(V_{d}^{k}\right)^{T}.$$

为了将SVD的结果分为两部分，使得$Y^k \approx W^k C^k$，我们可以取：

$$W^{k}=U_{d}^{k}\left(\Sigma_{d}^{k}\right)^{\frac{1}{2}}, \quad C^{k}=\left(\Sigma_{d}^{k}\right)^{\frac{1}{2}} (V_{d}^{k})^T.$$

值得注意的是，在我们对矩阵做SVD时，我们总希望矩阵是well-conditioned，否则分解效果可能会很差。我们发现，因为$Y^k_{i,j} = \log \left(\frac{A_{i, j}^{k}}{\sum_{i^{\prime}} A_{i^{\prime}, j}^{k}}\right)-\log (\frac{\lambda}{N})$，倘若节点对$(i,j)$之间的$k$阶转移概率对节点$j$来说占比较少，那么$\frac{A_{i, j}^{k}}{\sum_{i^{\prime}} A_{i^{\prime}, j}^{k}}$可能非常小，甚至会趋于零。在这种情况下$Y_{i,j}^k$可能非常小，甚至趋于负无穷，那么自然矩阵会变得ill-conditioned。因此一个直观的做法就是将$Y$矩阵中为复数的元素直接取零。这背后的intuition就是，如果节点对$(i,j)$之间的$k$阶转移概率对节点$j$来说占比非常少（对节点$j$来说节点$i$不重要），那么我们一概将其赋值为一个足够小的固定值$\frac{\lambda}{N}$。因此这个操作也不会对结果产生过大的影响。这个思路有点像Naive Bayes里的Laplace smoothing，用来防止翻车。

至此，我们对GraRep推导完毕。这篇文章极大的依赖[8]中的结果，来利用矩阵分解算法优化模型。因此，该算法被普遍认为是基于矩阵分解的算法。不过，这篇文章事实上也可以基于梯度下降进行求解，毕竟它的目标函数和skip-gram类的模型是基本一致的。

### 3.3 Implementation

在[GraRep.py](https://github.com/siqim/Machine-Learning-with-Graphs/blob/main/examples/0_MatrixFactorization/GraRep.py)中，我们实现了GraRep，并且在[Wikipedia](https://snap.stanford.edu/node2vec/)和[ogbn-arxiv](https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-arxiv)数据集上评估了该模型实现。该模型实现可在Wikipedia数据集上取得约0.55的Mirco-F1，在ogbn-arxiv数据集上取得约0.62的Accuracy。我们对Wikipedia效果的评估希望同[cogdl leaderboard](https://github.com/THUDM/cogdl#unsupervised-multi-label-node-classification)尽可能接近，即，用90%的标注数据进行训练，用10%的标注数据进行测试。对ogbn-arxiv的评估则完全基于ogb给出的划分和evaluator。

有趣的是，当我们在推导中去掉条件概率，使目标函数从NCE完全转变为NS时，我们可以在Wikipedia数据集中取得约0.59的Micro-F1，在ogbn-arxiv数据集上取得约0.61的Accuracy。

### 3.4 Summary
这篇文章提供的思路非常直接，就是希望重建$A^k$。为了做到这一点，该算法极大的利用了[7]中的目标函数和[8]中的求解思路。虽然该算法达到了可观的效果，但不得不给人一种大自然的搬运工的感觉。

总体来说，该模型的优点有：

- （理论上）能够捕捉任意阶的邻居连接关系
- 不同阶的邻居关系被映射到不同的子空间中，使得模型能够区分出不同阶的邻居

该模型的局限性有：

- 计算$A^k$和分解$Y^k$的代价极大，同时节点间也无参数共享，因此无法扩展至大规模图数据上
- 模型推导过程中有较强的$\vec{w_i} \cdot \vec{c_j}$独立假设，现实中可能无法达到，导致结果非最优
- low-rank SVD逼近能力有限，模型表达能力有限
- 尽管将一个节点的表达分为了$\vec{w_i}$和$\vec{c_i}$，但没有很好的利用$\vec{c_i}$
- 无法直接利用节点的特征


## 4. HOPE

## 5. References
[1] Amr Ahmed, Nino Shervashidze, Shravan Narayanamurthy,
Vanja Josifovski, and Alexander J Smola. Distributed
large-scale natural graph factorization. In
Proceedings of the 22nd international conference on
World Wide Web, pages 37–48, 2013.

[2] Shaosheng Cao, Wei Lu, and Qiongkai Xu. Grarep:
Learning graph representations with global structural
information. In Proceedings of the 24th ACM international
on conference on information and knowledge
management, pages 891–900, 2015.

[3] Mingdong Ou, Peng Cui, Jian Pei, Ziwei Zhang,
and Wenwu Zhu. Asymmetric transitivity preserving
graph embedding. In Proceedings of the 22nd ACM
SIGKDD international conference on Knowledge discovery
and data mining, pages 1105–1114, 2016.

[4] Wei-Lin Chiang, Xuanqing Liu, Si Si, Yang Li, Samy
Bengio, and Cho-Jui Hsieh. Cluster-gcn: An efficient
algorithm for training deep and large graph convolutional
networks. In Proceedings of the 25th ACM
SIGKDD International Conference on Knowledge Discovery
& Data Mining, pages 257–266, 2019.

[5] Bryan Perozzi, Rami Al-Rfou, and Steven Skiena.
Deepwalk: Online learning of social representations. In
Proceedings of the 20th ACM SIGKDD international
conference on Knowledge discovery and data mining,
pages 701–710, 2014.

[6] Jian Tang, Meng Qu, Mingzhe Wang, Ming Zhang,
Jun Yan, and Qiaozhu Mei. Line: Large-scale information
network embedding. In Proceedings of the
24th international conference on world wide web, pages
1067–1077, 2015.

[7] M. U. Gutmann and A. Hyv ̈arinen.
Noise-contrastive esti-mation of unnormalized statistical models, with applicationsto natural image statistics.
The journal of machine learningresearch, 13(1):307–361, 2012.

[8] O. Levy and Y. Goldberg. Neural word embedding as
implicit matrix factorization. In NIPS, pages
2177-2185, 2014.